# QC (sub workflows) combined: UHS4 v0007
Using UHS4 version 0007

## Create Directories

In [ ]:
mkdir -p /shared/rti-shared/shared_data/post_qc/uhs1234/genotype/array/observed/0003/{wf_input,wf_output}
mkdir -p /shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs1234/genotype/array/observed/0003/{sex_check,relatedness}/

##  Set up config file for QC pipeline

Copy JSON file from previous run and modify settings. Then edit this config file to include the appropriate cohort information.

In [ ]:
cd /shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs1234/genotype/array/observed/0003/
cp /shared/biocloud_gwas_workflows/genotype_array_qc/sex_check/test/test_sex_check.json sex_check/uhs1234_afr_sex_check_wf.json
cp /shared/biocloud_gwas_workflows/genotype_array_qc/relatedness/test_relatedness_wf.json relatedness/uhs1234_afr_relatedness_wf.json
cp /shared/biocloud_gwas_workflows/genotype_array_qc/sex_check/test/test_sex_check.json sex_check/uhs1234_eur_sex_check_wf.json
cp /shared/biocloud_gwas_workflows/genotype_array_qc/relatedness/test_relatedness_wf.json relatedness/uhs1234_eur_relatedness_wf.json

# Edit wf config files

## Zip biocloud_gwas_workflows repo

In [ ]:
cd /shared/biocloud_gwas_workflows
git pull
git submodule update --init --recursive
git rev-parse HEAD > /shared/rti-shared/shared_data/post_qc/uhs1234/genotype/array/observed/0003/wf_input/git_hash.txt
cd /shared
zip \
    --exclude=*/var/* \
    --exclude=*.git/* \
    --exclude=*/test/* \
    --exclude=*/.idea/* \
    -r /shared/rti-shared/shared_data/post_qc/uhs1234/genotype/array/observed/0003/wf_input/biocloud_gwas_workflows.zip \
    biocloud_gwas_workflows/

cd /shared/rti-shared/shared_data/post_qc/uhs1234/genotype/array/observed/0003/

## Submit jobs

In [ ]:
# Open session in terminal 1
ssh -i ~/.ssh/gwas_rsa -L localhost:8000:localhost:8000 ec2-user@54.174.185.7

# Submit jobs in terminal 2

# AFR relatedness
curl -X POST "http://localhost:8000/api/workflows/v1" -H "accept: application/json" \
    -F "workflowSource=@/shared/biocloud_gwas_workflows/genotype_array_qc/relatedness/relatedness_wf.wdl" \
    -F "workflowInputs=@/shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs1234/genotype/array/observed/0003/relatedness/uhs1234_afr_relatedness_wf.json" \
    -F "workflowDependencies=@/shared/rti-shared/shared_data/post_qc/uhs1234/genotype/array/observed/0003/wf_input/biocloud_gwas_workflows.zip"
echo ""
afr_relate=9af1abbc-742c-427a-a1de-8b002f1fa7d7

# AFR sex_check
curl -X POST "http://localhost:8000/api/workflows/v1" -H "accept: application/json" \
    -F "workflowSource=@/shared/biocloud_gwas_workflows/genotype_array_qc/sex_check/sex_check_wf.wdl" \
    -F "workflowInputs=@/shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs1234/genotype/array/observed/0003/sex_check/uhs1234_afr_sex_check_wf.json" \
    -F "workflowDependencies"=@/shared/rti-shared/shared_data/post_qc/uhs1234/genotype/array/observed/0003/wf_input/biocloud_gwas_workflows.zip
echo ""
afr_sex=a857e408-35a2-41ae-9e5a-baf1b96cbfda

# EUR relatedness
curl -X POST "http://localhost:8000/api/workflows/v1" -H "accept: application/json" \
    -F "workflowSource=@/shared/biocloud_gwas_workflows/genotype_array_qc/relatedness/relatedness_wf.wdl" \
    -F "workflowInputs=@/shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs1234/genotype/array/observed/0003/relatedness/uhs1234_eur_relatedness_wf.json" \
    -F "workflowDependencies"=@/shared/rti-shared/shared_data/post_qc/uhs1234/genotype/array/observed/0003/wf_input/biocloud_gwas_workflows.zip
echo ""
eur_relate=40f72a80-1164-4694-9594-fdf7d1dcc43c



# EUR sex_check
curl -X POST "http://localhost:8000/api/workflows/v1" -H "accept: application/json" \
    -F "workflowSource=@/shared/biocloud_gwas_workflows/genotype_array_qc/sex_check/sex_check_wf.wdl" \
    -F "workflowInputs=@/shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs1234/genotype/array/observed/0003/sex_check/uhs1234_eur_sex_check_wf.json" \
    -F "workflowDependencies"=@/shared/rti-shared/shared_data/post_qc/uhs1234/genotype/array/observed/0003/wf_input/biocloud_gwas_workflows.zip
echo ""
eur_sex=65c8669f-7748-4d15-bf4b-66b84f9e5983


# Monitor job in terminal 1
#tail -f /tmp/cromwell-server.log

# check job status in terminal 2
for job in {$afr_relate,$afr_sex,$eur_relate,$eur_sex}; do
    curl -X GET "http://localhost:8000/api/workflows/v1/${job}/status"   
    echo ""
done

## Get final outputs

In terminal with credentials for accessing rti-code S3.

### Relatedness

In [ ]:
cd /shared/rti-shared/shared_data/post_qc/uhs1234/genotype/array/observed/0003/wf_output/
mkdir relatedness
cd relatedness

afr_job=9af1abbc-742c-427a-a1de-8b002f1fa7d7
eur_job=40f72a80-1164-4694-9594-fdf7d1dcc43c

# Download output from Swagger UI.
curl -X GET "http://localhost:8000/api/workflows/v1/${afr_job}/outputs" -H "accept: application/json" \
  > ${afr_job}_final_outputs.json
curl -X GET "http://localhost:8000/api/workflows/v1/${eur_job}/outputs" -H "accept: application/json" \
  > ${eur_job}_final_outputs.json

# Copy final outputs json to S3
aws s3 cp ${job}_final_outputs.json \
  s3://rti-shared/shared_data/post_qc/uhs1234/genotype/array/observed/0003/wf_output/

# Copy final qc files from cromwell-output to final location
python - <<EOF
import json
import os
import re

job = "$job"
dir = "rti-shared/shared_data/post_qc/uhs1234/genotype/array/observed/0003/afr"

def traverse(o, tree_types=(list, tuple)):
    if isinstance(o, tree_types):
        for value in o:
            for subvalue in traverse(value, tree_types):
                yield subvalue
    else:
        yield o

with open(job + '_final_outputs.json') as f:
    outputs = json.load(f)
    outputs = outputs["outputs"]
    for key in outputs:
        if (type(outputs[key]) == list):
            for value in traverse(outputs[key]):
                if (str(value)[0:2] == "s3"):
                    message = "aws s3 cp {} s3://{}/wf_output/".format(value, dir)
                    os.system(message)
        else:
            if (str(outputs[key])[0:2] == "s3"):
                message = "aws s3 cp {} s3://{}/wf_output/".format(outputs[key], dir)
                os.system(message)

# Move and rename final genotype plink file set
with open('/shared/' + dir + '/wf_output/' + job + '_final_outputs.json') as f:
    outputs = json.load(f)
    outputs = outputs["outputs"]
    for ext in ["bed", "bim", "fam"]:
        final = "genotype_array_qc_wf.final_qc_{}".format(ext)
        for original in outputs[final]:
            fileName = re.sub(r'.+/', '', original)
            new = re.sub(r'uhs1234.YRI.+\.', 'afr/uhs1234.', fileName)
            new = re.sub(r'uhs1234.CEU.+\.', 'eur/uhs1234.', new)
            new = re.sub(r'uhs1234.CHB.+\.', 'amr/uhs1234.', new)
            final = "s3://{}/wf_output/{}".format(dir, new)
            os.system("aws s3 cp {} {}".format(original, final))

EOF

```
  821 uhs1234_afr_relatedness.final.related_samples.remove
  279 uhs1234_eur_relatedness.final.related_samples.remove 
``` 

### Sex check

In [ ]:
cd /shared/rti-shared/shared_data/post_qc/uhs1234/genotype/array/observed/0003/wf_output/
mkdir sex_check
cd sex_check


eur_job=65c8669f-7748-4d15-bf4b-66b84f9e5983
afr_job=a857e408-35a2-41ae-9e5a-baf1b96cbfda




# Download output from Swagger UI.
curl -X GET "http://localhost:8000/api/workflows/v1/${afr_job}/outputs" -H "accept: application/json" \
  > ${afr_job}_final_outputs.json
curl -X GET "http://localhost:8000/api/workflows/v1/${eur_job}/outputs" -H "accept: application/json" \
  > ${eur_job}_final_outputs.json


```
  790 uhs1234_afr_sex_check.sexcheck.sexcheck.remove
  855 uhs1234_eur_sex_check.sexcheck.sexcheck.remove
```

# QC (sub workflows) 0008 each cohort individually
Run the QC sub wf on each cohort individually.

```
s3://rti-hiv/rti-midas-data/studies/hiv/observed/final/uhs1.{aa,ea}.$ext.gz
s3://rti-shared/shared_data/post_qc/uhs2/genotype/array/observed/0001/{afr,eur}/uhs2_{afr,eur}.$ext.gz
s3://rti-hiv/rti-midas-data/studies/hiv/observed/final/uhs3.{aa,ea}.V1-2.$ext.gz
s3://rti-hiv/rti-midas-data/studies/hiv/observed/final/uhs3.{aa,ea}.V1-3.$ext.gz
s3://rti-shared/rti-midas-data/studies/uhs4/observed/genotypes/final/uhs4.merged2.{aa,ea}.$ext.gz
```

Note that I need to use the updated UHS2 results that contain the sex information.

## UHS1

### Create Directories

In [ ]:
mkdir -p /shared/rti-shared/shared_data/post_qc/uhs1/genotype/array/observed/0008/{wf_input,wf_output} # UHS1 AFR and EUR
mkdir -p /shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs1/genotype/array/observed/0008/{sex_check,relatedness}/

###  Set up config file for QC pipeline

Copy JSON file from previous run and modify settings. Then edit this config file to include the appropriate cohort information.

In [ ]:
cd /shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs1/genotype/array/observed/0008/

# sex
cp /shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs1234/genotype/array/observed/0004/sex_check/uhs1234_afr_sex_check_wf.json \
    sex_check/uhs1_afr_sex_check_wf.json
cp /shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs1234/genotype/array/observed/0004/sex_check/uhs1234_eur_sex_check_wf.json \
    sex_check/uhs1_eur_sex_check_wf.json

# relatedness
cp /shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs1234/genotype/array/observed/0004/relatedness/uhs1234_afr_relatedness_wf.json \
    relatedness/uhs1_afr_relatedness_wf.json
cp /shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs1234/genotype/array/observed/0004/relatedness/uhs1234_eur_relatedness_wf.json \
    relatedness/uhs1_eur_relatedness_wf.json

# Edit wf config files

### Zip biocloud_gwas_workflows repo

In [ ]:
cd /shared/biocloud_gwas_workflows
git pull
git submodule update --init --recursive
git rev-parse HEAD > /shared/rti-shared/shared_data/post_qc/uhs1/genotype/array/observed/0008/wf_input/git_hash.txt
cd /shared
zip \
    --exclude=*/var/* \
    --exclude=*.git/* \
    --exclude=*/test/* \
    --exclude=*/.idea/* \
    -r /shared/rti-shared/shared_data/post_qc/uhs1/genotype/array/observed/0008/wf_input/biocloud_gwas_workflows.zip \
    biocloud_gwas_workflows/

cd /shared/rti-shared/shared_data/post_qc/uhs1/genotype/array/observed/0008

### Submit jobs

In [ ]:
# Open session in terminal 1
ssh -i ~/.ssh/gwas_rsa -L localhost:8000:localhost:8000 ec2-user@54.174.185.7

# Submit jobs in terminal 2


# AFR sex_check
curl -X POST "http://localhost:8000/api/workflows/v1" -H "accept: application/json" \
    -F "workflowSource=@/shared/biocloud_gwas_workflows/genotype_array_qc/sex_check/sex_check_wf.wdl" \
    -F "workflowInputs=@/shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs1/genotype/array/observed/0008/sex_check/uhs1_afr_sex_check_wf.json" \
    -F "workflowDependencies=@/shared/rti-shared/shared_data/post_qc/uhs1/genotype/array/observed/0008/wf_input/biocloud_gwas_workflows.zip"
echo ""
afr_sex=

# EUR sex_check
curl -X POST "http://localhost:8000/api/workflows/v1" -H "accept: application/json" \
    -F "workflowSource=@/shared/biocloud_gwas_workflows/genotype_array_qc/sex_check/sex_check_wf.wdl" \
    -F "workflowInputs=@/shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs1/genotype/array/observed/0008/sex_check/uhs1_eur_sex_check_wf.json" \
    -F "workflowDependencies=@/shared/rti-shared/shared_data/post_qc/uhs1/genotype/array/observed/0008/wf_input/biocloud_gwas_workflows.zip"
echo ""
eur_sex=

# AFR relatedness
curl -X POST "http://localhost:8000/api/workflows/v1" -H "accept: application/json" \
    -F "workflowSource=@/shared/biocloud_gwas_workflows/genotype_array_qc/relatedness/relatedness_wf.wdl" \
    -F "workflowInputs=@/shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs1/genotype/array/observed/0008/relatedness/uhs1_afr_relatedness_wf.json" \
    -F "workflowDependencies=@/shared/rti-shared/shared_data/post_qc/uhs1/genotype/array/observed/0008/wf_input/biocloud_gwas_workflows.zip"
echo ""

# EUR relatedness
curl -X POST "http://localhost:8000/api/workflows/v1" -H "accept: application/json" \
    -F "workflowSource=@/shared/biocloud_gwas_workflows/genotype_array_qc/relatedness/relatedness_wf.wdl" \
    -F "workflowInputs=@/shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs1/genotype/array/observed/0008/relatedness/uhs1_eur_relatedness_wf.json" \
    -F "workflowDependencies=@/shared/rti-shared/shared_data/post_qc/uhs1/genotype/array/observed/0008/wf_input/biocloud_gwas_workflows.zip"
echo ""

# Monitor job in terminal 1
#tail -f /tmp/cromwell-server.log

afr_relate=45d9cade-fa50-463c-beb0-825e3d9f3b0f
eur_relate=cbc07f17-2644-4c71-907d-7eaec2425dfb

# check job status in terminal 2
for job in {$afr_relate,$afr_sex,$eur_relate,$eur_sex}; do
    curl -X GET "http://localhost:8000/api/workflows/v1/${job}/status"   
    echo ""
done

```
{"status":"Succeeded","id":"45d9cade-fa50-463c-beb0-825e3d9f3b0f"}
{"status":"Succeeded","id":"cbc07f17-2644-4c71-907d-7eaec2425dfb"}
```

In [ ]:
cd /shared/rti-shared/shared_data/post_qc/uhs1/genotype/array/observed/0008/wf_output
mkdir sex_check relatedness
cd relatedness

echo $eur_relate $afr_relate


# Download output from Swagger UI.
curl -X GET "http://localhost:8000/api/workflows/v1/${afr_relate}/outputs" -H "accept: application/json" \
  > ${afr_relate}_final_outputs.json
curl -X GET "http://localhost:8000/api/workflows/v1/${eur_relate}/outputs" -H "accept: application/json" \
  > ${eur_relate}_final_outputs.json

aws s3 cp s3://rti-cromwell-output/cromwell-execution/relatedness_wf/cbc07f17-2644-4c71-907d-7eaec2425dfb/call-restore_pedigree_ids/uhs1_eur_relatedness.final.related_samples.remove .
aws s3 cp s3://rti-cromwell-output/cromwell-execution/relatedness_wf/45d9cade-fa50-463c-beb0-825e3d9f3b0f/call-restore_pedigree_ids/uhs1_afr_relatedness.final.related_samples.remove .    
aws s3 cp s3://rti-cromwell-output/cromwell-execution/relatedness_wf/cbc07f17-2644-4c71-907d-7eaec2425dfb/call-remove_fam_pedigree/uhs1_eur_relatedness.noped.idmap.fam .
aws s3 cp s3://rti-cromwell-output/cromwell-execution/relatedness_wf/45d9cade-fa50-463c-beb0-825e3d9f3b0f/call-final_get_relateds/KING.king_kinship_wf/6633dddc-5ee6-4903-bc5c-41c9fe0af177/call-prune_related_samples/uhs1_afr_relatedness.final.king.pruned.annotated.k0 .    
    
wc -l *e
#3   uhs1_eur_relatedness.final.related_samples.remove
#102 uhs1_afr_relatedness.final.related_samples.remove


cat *e
#381786@1054736429 381786@1054736429
#433346@1054752818 433346@1054752818
#533711@1054697996 533711@1054697996

awk '$9=="2nd_degree_relative"' uhs1_afr_relatedness.final.king.pruned.annotated.k0  | wc -l
#58
awk '$9=="3+_degree_relative"' uhs1_afr_relatedness.final.king.pruned.annotated.k0  | wc -l
#66

## UHS2

### Create Directories

In [ ]:
mkdir -p /shared/rti-shared/shared_data/post_qc/uhs2/genotype/array/observed/0008/{wf_input,wf_output} # UHS2 AFR and EUR
mkdir -p /shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs2/genotype/array/observed/0008/{sex_check,relatedness}/

###  Set up config file for QC pipeline

Copy JSON file from previous run and modify settings. Then edit this config file to include the appropriate cohort information.

In [ ]:
cd /shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs2/genotype/array/observed/0008/

# sex
cp /shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs1/genotype/array/observed/0008/sex_check/uhs1_afr_sex_check_wf.json \
    sex_check/uhs2_afr_sex_check_wf.json
cp /shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs1/genotype/array/observed/0008/sex_check/uhs1_eur_sex_check_wf.json \
    sex_check/uhs2_eur_sex_check_wf.json

# relatedness
cp /shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs1/genotype/array/observed/0008/relatedness/uhs1_afr_relatedness_wf.json \
    relatedness/uhs2_afr_relatedness_wf.json
cp /shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs1/genotype/array/observed/0008/relatedness/uhs1_eur_relatedness_wf.json \
    relatedness/uhs2_eur_relatedness_wf.json

# Edit wf config files

### Zip biocloud_gwas_workflows repo

In [ ]:
cd /shared/biocloud_gwas_workflows
git pull
git submodule update --init --recursive
git rev-parse HEAD > /shared/rti-shared/shared_data/post_qc/uhs2/genotype/array/observed/0008/wf_input/git_hash.txt
cd /shared
zip \
    --exclude=*/var/* \
    --exclude=*.git/* \
    --exclude=*/test/* \
    --exclude=*/.idea/* \
    -r /shared/rti-shared/shared_data/post_qc/uhs2/genotype/array/observed/0008/wf_input/biocloud_gwas_workflows.zip \
    biocloud_gwas_workflows/

cd /shared/rti-shared/shared_data/post_qc/uhs2/genotype/array/observed/0008

### Submit jobs

In [ ]:
# Open session in terminal 1
ssh -i ~/.ssh/gwas_rsa -L localhost:8000:localhost:8000 ec2-user@54.174.185.7

# Submit jobs in terminal 2

# AFR sex_check
curl -X POST "http://localhost:8000/api/workflows/v1" -H "accept: application/json" \
    -F "workflowSource=@/shared/biocloud_gwas_workflows/genotype_array_qc/sex_check/sex_check_wf.wdl" \
    -F "workflowInputs=@/shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs2/genotype/array/observed/0008/sex_check/uhs2_afr_sex_check_wf.json" \
    -F "workflowDependencies=@/shared/rti-shared/shared_data/post_qc/uhs2/genotype/array/observed/0008/wf_input/biocloud_gwas_workflows.zip"
echo ""

# EUR sex_check
curl -X POST "http://localhost:8000/api/workflows/v1" -H "accept: application/json" \
    -F "workflowSource=@/shared/biocloud_gwas_workflows/genotype_array_qc/sex_check/sex_check_wf.wdl" \
    -F "workflowInputs=@/shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs2/genotype/array/observed/0008/sex_check/uhs2_eur_sex_check_wf.json" \
    -F "workflowDependencies=@/shared/rti-shared/shared_data/post_qc/uhs2/genotype/array/observed/0008/wf_input/biocloud_gwas_workflows.zip"
echo ""


# AFR relatedness
curl -X POST "http://localhost:8000/api/workflows/v1" -H "accept: application/json" \
    -F "workflowSource=@/shared/biocloud_gwas_workflows/genotype_array_qc/relatedness/relatedness_wf.wdl" \
    -F "workflowInputs=@/shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs2/genotype/array/observed/0008/relatedness/uhs2_afr_relatedness_wf.json" \
    -F "workflowDependencies=@/shared/rti-shared/shared_data/post_qc/uhs2/genotype/array/observed/0008/wf_input/biocloud_gwas_workflows.zip"
echo ""

# EUR relatedness
curl -X POST "http://localhost:8000/api/workflows/v1" -H "accept: application/json" \
    -F "workflowSource=@/shared/biocloud_gwas_workflows/genotype_array_qc/relatedness/relatedness_wf.wdl" \
    -F "workflowInputs=@/shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs2/genotype/array/observed/0008/relatedness/uhs2_eur_relatedness_wf.json" \
    -F "workflowDependencies=@/shared/rti-shared/shared_data/post_qc/uhs2/genotype/array/observed/0008/wf_input/biocloud_gwas_workflows.zip"

afr_sex=
eur_sex=
afr_relate=99da8cf3-b9d5-4903-a5ac-193a65e6555b
eur_relate=c917bec6-dd16-4f84-a289-6f8e889b3a6e


# check job status in terminal 2
for job in {$afr_relate,$afr_sex,$eur_relate,$eur_sex}; do
    curl -X GET "http://localhost:8000/api/workflows/v1/${job}/status"   
    echo ""
done

```
{"status":"Succeeded","id":"99da8cf3-b9d5-4903-a5ac-193a65e6555b"}
{"status":"Succeeded","id":"c917bec6-dd16-4f84-a289-6f8e889b3a6e"}
```

In [ ]:
cd /shared/rti-shared/shared_data/post_qc/uhs2/genotype/array/observed/0008/wf_output
mkdir sex_check relatedness
cd relatedness

echo $eur_relate $afr_relate


# Download output from Swagger UI.
curl -X GET "http://localhost:8000/api/workflows/v1/${afr_relate}/outputs" -H "accept: application/json" \
  > ${afr_relate}_final_outputs.json
curl -X GET "http://localhost:8000/api/workflows/v1/${eur_relate}/outputs" -H "accept: application/json" \
  > ${eur_relate}_final_outputs.json

aws s3 cp s3://rti-cromwell-output/cromwell-execution/relatedness_wf/99da8cf3-b9d5-4903-a5ac-193a65e6555b/call-restore_pedigree_ids/uhs2_afr_relatedness.final.related_samples.remove . 
aws s3 cp s3://rti-cromwell-output/cromwell-execution/relatedness_wf/99da8cf3-b9d5-4903-a5ac-193a65e6555b/call-remove_fam_pedigree/uhs2_afr_relatedness.noped.idmap.fam .  
aws s3 cp s3://rti-cromwell-output/cromwell-execution/relatedness_wf/99da8cf3-b9d5-4903-a5ac-193a65e6555b/call-final_get_relateds/KING.king_kinship_wf/8d6850fa-80ac-4195-b8ce-53288fc15dfc/call-prune_related_samples/uhs2_afr_relatedness.final.king.pruned.annotated.k0 .
aws s3 cp s3://rti-cromwell-output/cromwell-execution/relatedness_wf/c917bec6-dd16-4f84-a289-6f8e889b3a6e/call-restore_pedigree_ids/uhs2_eur_relatedness.final.related_samples.remove .
aws s3 cp s3://rti-cromwell-output/cromwell-execution/relatedness_wf/c917bec6-dd16-4f84-a289-6f8e889b3a6e/call-remove_fam_pedigree/uhs2_eur_relatedness.noped.idmap.fam .
aws s3 cp s3://rti-cromwell-output/cromwell-execution/relatedness_wf/c917bec6-dd16-4f84-a289-6f8e889b3a6e/call-final_get_relateds/KING.king_kinship_wf/f0997f2d-a0d9-4f46-aa4b-a1987df63dff/call-prune_related_samples/uhs2_eur_relatedness.final.king.pruned.annotated.k0 .
    
wc -l *remove
#  19 uhs2_afr_relatedness.final.related_samples.remove
#   0 uhs2_eur_relatedness.final.related_samples.remove

awk '$9=="2nd_degree_relative"' uhs2_afr_relatedness.final.king.pruned.annotated.k0  | wc -l
#10
awk '$9=="3+_degree_relative"' uhs2_afr_relatedness.final.king.pruned.annotated.k0  | wc -l
#12

## UHS3_v1-2

### Create Directories

In [ ]:
mkdir -p /shared/rti-shared/shared_data/post_qc/uhs3_v1-2/genotype/array/observed/0008/{wf_input,wf_output} # UHS3_v1-2 AFR and EUR
mkdir -p /shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs3_v1-2/genotype/array/observed/0008/{sex_check,relatedness}/

###  Set up config file for QC pipeline

Copy JSON file from previous run and modify settings. Then edit this config file to include the appropriate cohort information.

In [ ]:
cd /shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs3_v1-2/genotype/array/observed/0008/

# sex
cp /shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs1/genotype/array/observed/0008/sex_check/uhs1_afr_sex_check_wf.json \
    sex_check/uhs3_v1-2_afr_sex_check_wf.json
cp /shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs1/genotype/array/observed/0008/sex_check/uhs1_eur_sex_check_wf.json \
    sex_check/uhs3_v1-2_eur_sex_check_wf.json

# relatedness
cp /shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs1/genotype/array/observed/0008/relatedness/uhs1_afr_relatedness_wf.json \
    relatedness/uhs3_v1-2_afr_relatedness_wf.json
cp /shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs1/genotype/array/observed/0008/relatedness/uhs1_eur_relatedness_wf.json \
    relatedness/uhs3_v1-2_eur_relatedness_wf.json

# Edit wf config files

### Zip biocloud_gwas_workflows repo

In [ ]:
cd /shared/biocloud_gwas_workflows
git pull
git submodule update --init --recursive
git rev-parse HEAD > /shared/rti-shared/shared_data/post_qc/uhs3_v1-2/genotype/array/observed/0008/wf_input/git_hash.txt
cd /shared
zip \
    --exclude=*/var/* \
    --exclude=*.git/* \
    --exclude=*/test/* \
    --exclude=*/.idea/* \
    -r /shared/rti-shared/shared_data/post_qc/uhs3_v1-2/genotype/array/observed/0008/wf_input/biocloud_gwas_workflows.zip \
    biocloud_gwas_workflows/

cd /shared/rti-shared/shared_data/post_qc/uhs3_v1-2/genotype/array/observed/0008

### Submit jobs

In [ ]:
# Open session in terminal 1
ssh -i ~/.ssh/gwas_rsa -L localhost:8000:localhost:8000 ec2-user@54.174.185.7

# Submit jobs in terminal 2

# AFR sex_check
curl -X POST "http://localhost:8000/api/workflows/v1" -H "accept: application/json" \
    -F "workflowSource=@/shared/biocloud_gwas_workflows/genotype_array_qc/sex_check/sex_check_wf.wdl" \
    -F "workflowInputs=@/shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs3_v1-2/genotype/array/observed/0008/sex_check/uhs3_v1-2_afr_sex_check_wf.json" \
    -F "workflowDependencies=@/shared/rti-shared/shared_data/post_qc/uhs3_v1-2/genotype/array/observed/0008/wf_input/biocloud_gwas_workflows.zip"
echo ""

# EUR sex_check
curl -X POST "http://localhost:8000/api/workflows/v1" -H "accept: application/json" \
    -F "workflowSource=@/shared/biocloud_gwas_workflows/genotype_array_qc/sex_check/sex_check_wf.wdl" \
    -F "workflowInputs=@/shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs3_v1-2/genotype/array/observed/0008/sex_check/uhs3_v1-2_eur_sex_check_wf.json" \
    -F "workflowDependencies=@/shared/rti-shared/shared_data/post_qc/uhs3_v1-2/genotype/array/observed/0008/wf_input/biocloud_gwas_workflows.zip"
echo ""
_____________________________________________________________________________________________________________________________________________________________________

# AFR relatedness
curl -X POST "http://localhost:8000/api/workflows/v1" -H "accept: application/json" \
    -F "workflowSource=@/shared/biocloud_gwas_workflows/genotype_array_qc/relatedness/relatedness_wf.wdl" \
    -F "workflowInputs=@/shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs3_v1-2/genotype/array/observed/0008/relatedness/uhs3_v1-2_afr_relatedness_wf.json" \
    -F "workflowDependencies=@/shared/rti-shared/shared_data/post_qc/uhs3_v1-2/genotype/array/observed/0008/wf_input/biocloud_gwas_workflows.zip"
echo ""

# EUR relatedness
curl -X POST "http://localhost:8000/api/workflows/v1" -H "accept: application/json" \
    -F "workflowSource=@/shared/biocloud_gwas_workflows/genotype_array_qc/relatedness/relatedness_wf.wdl" \
    -F "workflowInputs=@/shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs3_v1-2/genotype/array/observed/0008/relatedness/uhs3_v1-2_eur_relatedness_wf.json" \
    -F "workflowDependencies=@/shared/rti-shared/shared_data/post_qc/uhs3_v1-2/genotype/array/observed/0008/wf_input/biocloud_gwas_workflows.zip"

afr_sex=
eur_sex=

afr_relate=b78c7c55-645e-40fc-95d6-07cc0fdb7a93
eur_relate=71795a5a-4ed6-48bd-804a-68acfd62d811

# Monitor job in terminal 1
#tail -f /tmp/cromwell-server.log

# check job status in terminal 2
for job in {$afr_relate,$afr_sex,$eur_relate,$eur_sex}; do
    curl -X GET "http://localhost:8000/api/workflows/v1/${job}/status"   
    echo ""
done

```
{"status":"Succeeded","id":"b78c7c55-645e-40fc-95d6-07cc0fdb7a93"}
```

In [ ]:
cd /shared/rti-shared/shared_data/post_qc/uhs3_1-2/genotype/array/observed/0008/wf_output
mkdir sex_check relatedness
cd relatedness

echo $eur_relate $afr_relate


# Download output from Swagger UI.
curl -X GET "http://localhost:8000/api/workflows/v1/${afr_relate}/outputs" -H "accept: application/json" \
  > ${afr_relate}_final_outputs.json
curl -X GET "http://localhost:8000/api/workflows/v1/${eur_relate}/outputs" -H "accept: application/json" \
  > ${eur_relate}_final_outputs.json

aws s3 cp s3://rti-cromwell-output/cromwell-execution/relatedness_wf/b78c7c55-645e-40fc-95d6-07cc0fdb7a93/call-restore_pedigree_ids/uhs3_v1-2_afr_relatedness.final.related_samples.remove .
aws s3 cp s3://rti-cromwell-output/cromwell-execution/relatedness_wf/b78c7c55-645e-40fc-95d6-07cc0fdb7a93/call-remove_fam_pedigree/uhs3_v1-2_afr_relatedness.noped.idmap.fam .
aws s3 cp s3://rti-cromwell-output/cromwell-execution/relatedness_wf/b78c7c55-645e-40fc-95d6-07cc0fdb7a93/call-final_get_relateds/KING.king_kinship_wf/674fa062-0d04-47c2-aca6-6e8a8a48d784/call-prune_related_samples/uhs3_v1-2_afr_relatedness.final.king.pruned.annotated.k0 .
    
wc -l *remove
#4 uhs3_v1-2_afr_relatedness.final.related_samples.remove

awk '$9=="2nd_degree_relative"' uhs3_v1-2_afr_relatedness.final.king.pruned.annotated.k0  | wc -l
#1
awk '$9=="3+_degree_relative"' uhs3_v1-2_afr_relatedness.final.king.pruned.annotated.k0  | wc -l
#15



## UHS3_v1-3

### Create Directories

In [ ]:
mkdir -p /shared/rti-shared/shared_data/post_qc/uhs3_v1-3/genotype/array/observed/0008/{wf_input,wf_output} # UHS3_v1-3 AFR and EUR
mkdir -p /shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs3_v1-3/genotype/array/observed/0008/{sex_check,relatedness}/

###  Set up config file for QC pipeline

Copy JSON file from previous run and modify settings. Then edit this config file to include the appropriate cohort information.

In [ ]:
cd /shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs3_v1-3/genotype/array/observed/0008/

# sex
cp /shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs1/genotype/array/observed/0008/sex_check/uhs1_afr_sex_check_wf.json \
    sex_check/uhs3_v1-3_afr_sex_check_wf.json
cp /shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs1/genotype/array/observed/0008/sex_check/uhs1_eur_sex_check_wf.json \
    sex_check/uhs3_v1-3_eur_sex_check_wf.json

# relatedness
cp /shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs1/genotype/array/observed/0008/relatedness/uhs1_afr_relatedness_wf.json \
    relatedness/uhs3_v1-3_afr_relatedness_wf.json
cp /shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs1/genotype/array/observed/0008/relatedness/uhs1_eur_relatedness_wf.json \
    relatedness/uhs3_v1-3_eur_relatedness_wf.json

# Edit wf config files

### Zip biocloud_gwas_workflows repo

In [ ]:
cd /shared/biocloud_gwas_workflows
git pull
git submodule update --init --recursive
git rev-parse HEAD > /shared/rti-shared/shared_data/post_qc/uhs3_v1-3/genotype/array/observed/0008/wf_input/git_hash.txt
cd /shared
zip \
    --exclude=*/var/* \
    --exclude=*.git/* \
    --exclude=*/test/* \
    --exclude=*/.idea/* \
    -r /shared/rti-shared/shared_data/post_qc/uhs3_v1-3/genotype/array/observed/0008/wf_input/biocloud_gwas_workflows.zip \
    biocloud_gwas_workflows/

cd /shared/rti-shared/shared_data/post_qc/uhs3_v1-3/genotype/array/observed/0008

### Submit jobs

In [ ]:
# Open session in terminal 1
ssh -i ~/.ssh/gwas_rsa -L localhost:8000:localhost:8000 ec2-user@54.174.185.7

# Submit jobs in terminal 2

# AFR sex_check
curl -X POST "http://localhost:8000/api/workflows/v1" -H "accept: application/json" \
    -F "workflowSource=@/shared/biocloud_gwas_workflows/genotype_array_qc/sex_check/sex_check_wf.wdl" \
    -F "workflowInputs=@/shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs3_v1-3/genotype/array/observed/0008/sex_check/uhs3_v1-3_afr_sex_check_wf.json" \
    -F "workflowDependencies=@/shared/rti-shared/shared_data/post_qc/uhs3_v1-3/genotype/array/observed/0008/wf_input/biocloud_gwas_workflows.zip"
echo ""

# EUR sex_check
curl -X POST "http://localhost:8000/api/workflows/v1" -H "accept: application/json" \
    -F "workflowSource=@/shared/biocloud_gwas_workflows/genotype_array_qc/sex_check/sex_check_wf.wdl" \
    -F "workflowInputs=@/shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs3_v1-3/genotype/array/observed/0008/sex_check/uhs3_v1-3_eur_sex_check_wf.json" \
    -F "workflowDependencies=@/shared/rti-shared/shared_data/post_qc/uhs3_v1-3/genotype/array/observed/0008/wf_input/biocloud_gwas_workflows.zip"
echo ""
_____________________________________________________________________________________________________________________________________________________________________

# AFR relatedness
curl -X POST "http://localhost:8000/api/workflows/v1" -H "accept: application/json" \
    -F "workflowSource=@/shared/biocloud_gwas_workflows/genotype_array_qc/relatedness/relatedness_wf.wdl" \
    -F "workflowInputs=@/shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs3_v1-3/genotype/array/observed/0008/relatedness/uhs3_v1-3_afr_relatedness_wf.json" \
    -F "workflowDependencies=@/shared/rti-shared/shared_data/post_qc/uhs3_v1-3/genotype/array/observed/0008/wf_input/biocloud_gwas_workflows.zip"
echo ""

# EUR relatedness
curl -X POST "http://localhost:8000/api/workflows/v1" -H "accept: application/json" \
    -F "workflowSource=@/shared/biocloud_gwas_workflows/genotype_array_qc/relatedness/relatedness_wf.wdl" \
    -F "workflowInputs=@/shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs3_v1-3/genotype/array/observed/0008/relatedness/uhs3_v1-3_eur_relatedness_wf.json" \
    -F "workflowDependencies=@/shared/rti-shared/shared_data/post_qc/uhs3_v1-3/genotype/array/observed/0008/wf_input/biocloud_gwas_workflows.zip"


afr_relate=5f08ed85-437c-4679-9b80-eb5b52635840
eur_relate=aee6d3fa-77ac-40dc-879f-35998740e063

# Monitor job in terminal 1
#tail -f /tmp/cromwell-server.log

# check job status in terminal 2
for job in {$afr_relate,$afr_sex,$eur_relate,$eur_sex}; do
    curl -X GET "http://localhost:8000/api/workflows/v1/${job}/status"   
    echo ""
done

```
{"status":"Succeeded","id":"5f08ed85-437c-4679-9b80-eb5b52635840"}
```

In [ ]:
cd /shared/rti-shared/shared_data/post_qc/uhs3_v1-3/genotype/array/observed/0008/wf_output
mkdir sex_check relatedness
cd relatedness

echo $eur_relate $afr_relate


# Download output from Swagger UI.
curl -X GET "http://localhost:8000/api/workflows/v1/${afr_relate}/outputs" -H "accept: application/json" \
  > ${afr_relate}_final_outputs.json
curl -X GET "http://localhost:8000/api/workflows/v1/${eur_relate}/outputs" -H "accept: application/json" \
  > ${eur_relate}_final_outputs.json

aws s3 cp s3://rti-cromwell-output/cromwell-execution/relatedness_wf/5f08ed85-437c-4679-9b80-eb5b52635840/call-final_get_relateds/KING.king_kinship_wf/2eb9d120-acc6-4ea4-9aa6-cee06e03543a/call-prune_related_samples/uhs3_v1-3_afr_relatedness.final.king.pruned.annotated.k0 .
aws s3 cp s3://rti-cromwell-output/cromwell-execution/relatedness_wf/5f08ed85-437c-4679-9b80-eb5b52635840/call-restore_pedigree_ids/uhs3_v1-3_afr_relatedness.final.related_samples.remove .
aws s3 cp s3://rti-cromwell-output/cromwell-execution/relatedness_wf/5f08ed85-437c-4679-9b80-eb5b52635840/call-remove_fam_pedigree/uhs3_v1-3_afr_relatedness.noped.idmap.fam .
    
wc -l *remove
1 uhs3_v1-3_afr_relatedness.final.related_samples.remove

awk '$9=="2nd_degree_relative"' uhs4_afr_relatedness.final.king.pruned.annotated.k0  | wc -l
# 24
awk '$9=="3+_degree_relative"' uhs4_afr_relatedness.final.king.pruned.annotated.k0  | wc -l
# 15


## UHS4

### Create Directories

In [ ]:
mkdir -p /shared/rti-shared/shared_data/post_qc/uhs4/genotype/array/observed/0008/{wf_input,wf_output} # UHS4 AFR and EUR
mkdir -p /shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs4/genotype/array/observed/0008/{sex_check,relatedness}/

###  Set up config file for QC pipeline

Copy JSON file from previous run and modify settings. Then edit this config file to include the appropriate cohort information.

In [ ]:
cd /shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs4/genotype/array/observed/0008/

# sex
cp /shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs1/genotype/array/observed/0008/sex_check/uhs1_afr_sex_check_wf.json \
    sex_check/uhs4_afr_sex_check_wf.json
cp /shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs1/genotype/array/observed/0008/sex_check/uhs1_eur_sex_check_wf.json \
    sex_check/uhs4_eur_sex_check_wf.json

# relatedness
cp /shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs1/genotype/array/observed/0008/relatedness/uhs1_afr_relatedness_wf.json \
    relatedness/uhs4_afr_relatedness_wf.json
cp /shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs1/genotype/array/observed/0008/relatedness/uhs1_eur_relatedness_wf.json \
    relatedness/uhs4_eur_relatedness_wf.json

# Edit wf config files

### Zip biocloud_gwas_workflows repo

In [ ]:
cd /shared/biocloud_gwas_workflows
git pull
git submodule update --init --recursive
git rev-parse HEAD > /shared/rti-shared/shared_data/post_qc/uhs4/genotype/array/observed/0008/wf_input/git_hash.txt
cd /shared
zip \
    --exclude=*/var/* \
    --exclude=*.git/* \
    --exclude=*/test/* \
    --exclude=*/.idea/* \
    -r /shared/rti-shared/shared_data/post_qc/uhs4/genotype/array/observed/0008/wf_input/biocloud_gwas_workflows.zip \
    biocloud_gwas_workflows/

cd /shared/rti-shared/shared_data/post_qc/uhs4/genotype/array/observed/0008

### Submit jobs

In [ ]:
# Open session in terminal 1
ssh -i ~/.ssh/gwas_rsa -L localhost:8000:localhost:8000 ec2-user@54.174.185.7

# Submit jobs in terminal 2

# AFR sex_check
curl -X POST "http://localhost:8000/api/workflows/v1" -H "accept: application/json" \
    -F "workflowSource=@/shared/biocloud_gwas_workflows/genotype_array_qc/sex_check/sex_check_wf.wdl" \
    -F "workflowInputs=@/shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs4/genotype/array/observed/0008/sex_check/uhs4_afr_sex_check_wf.json" \
    -F "workflowDependencies=@/shared/rti-shared/shared_data/post_qc/uhs4/genotype/array/observed/0008/wf_input/biocloud_gwas_workflows.zip"
echo ""

# EUR sex_check
curl -X POST "http://localhost:8000/api/workflows/v1" -H "accept: application/json" \
    -F "workflowSource=@/shared/biocloud_gwas_workflows/genotype_array_qc/sex_check/sex_check_wf.wdl" \
    -F "workflowInputs=@/shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs4/genotype/array/observed/0008/sex_check/uhs4_eur_sex_check_wf.json" \
    -F "workflowDependencies=@/shared/rti-shared/shared_data/post_qc/uhs4/genotype/array/observed/0008/wf_input/biocloud_gwas_workflows.zip"
echo ""
_____________________________________________________________________________________________________________________________________________________________________

# AFR relatedness
curl -X POST "http://localhost:8000/api/workflows/v1" -H "accept: application/json" \
    -F "workflowSource=@/shared/biocloud_gwas_workflows/genotype_array_qc/relatedness/relatedness_wf.wdl" \
    -F "workflowInputs=@/shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs4/genotype/array/observed/0008/relatedness/uhs4_afr_relatedness_wf.json" \
    -F "workflowDependencies=@/shared/rti-shared/shared_data/post_qc/uhs4/genotype/array/observed/0008/wf_input/biocloud_gwas_workflows.zip"
echo ""

# EUR relatedness
curl -X POST "http://localhost:8000/api/workflows/v1" -H "accept: application/json" \
    -F "workflowSource=@/shared/biocloud_gwas_workflows/genotype_array_qc/relatedness/relatedness_wf.wdl" \
    -F "workflowInputs=@/shared/bioinformatics/methods/rti-shared/shared_data/post_qc/uhs4/genotype/array/observed/0008/relatedness/uhs4_eur_relatedness_wf.json" \
    -F "workflowDependencies=@/shared/rti-shared/shared_data/post_qc/uhs4/genotype/array/observed/0008/wf_input/biocloud_gwas_workflows.zip"

afr_sex=
eur_sex=

eur_relate=6717954c-a6f5-43d5-9603-d0a2eea67a7b
afr_relate=5b0405dd-af65-4ad8-b025-a62680aa6235


# Monitor job in terminal 1
#tail -f /tmp/cromwell-server.log

# check job status in terminal 2
for job in {$afr_relate,$afr_sex,$eur_relate,$eur_sex}; do
    curl -X GET "http://localhost:8000/api/workflows/v1/${job}/status"   
    echo ""
done

```
{"status":"Succeeded","id":"5b0405dd-af65-4ad8-b025-a62680aa6235"}
{"status":"Succeeded","id":"6717954c-a6f5-43d5-9603-d0a2eea67a7b"}
```

### get results

In [ ]:
cd /shared/rti-shared/shared_data/post_qc/uhs4/genotype/array/observed/0008/wf_output
mkdir sex_check relatedness
cd relatedness

echo $eur_relate $afr_relate


# Download output from Swagger UI.
curl -X GET "http://localhost:8000/api/workflows/v1/${afr_relate}/outputs" -H "accept: application/json" \
  > ${afr_relate}_final_outputs.json
curl -X GET "http://localhost:8000/api/workflows/v1/${eur_relate}/outputs" -H "accept: application/json" \
  > ${eur_relate}_final_outputs.json

aws s3 cp s3://rti-cromwell-output/cromwell-execution/relatedness_wf/6717954c-a6f5-43d5-9603-d0a2eea67a7b/call-remove_fam_pedigree/uhs4_eur_relatedness.noped.idmap.fam .
aws s3 cp s3://rti-cromwell-output/cromwell-execution/relatedness_wf/6717954c-a6f5-43d5-9603-d0a2eea67a7b/call-restore_pedigree_ids/uhs4_eur_relatedness.final.related_samples.remove .
aws s3 cp s3://rti-cromwell-output/cromwell-execution/relatedness_wf/6717954c-a6f5-43d5-9603-d0a2eea67a7b/call-final_get_relateds/KING.king_kinship_wf/55253f75-8889-433c-b4b6-8473a7081a30/call-prune_related_samples/uhs4_eur_relatedness.final.king.pruned.annotated.k0 .
aws s3 cp s3://rti-cromwell-output/cromwell-execution/relatedness_wf/5b0405dd-af65-4ad8-b025-a62680aa6235/call-final_get_relateds/KING.king_kinship_wf/b0f5cd69-87bc-4398-9e6b-0e34654e0d5b/call-prune_related_samples/uhs4_afr_relatedness.final.king.pruned.annotated.k0 .
aws s3 cp s3://rti-cromwell-output/cromwell-execution/relatedness_wf/5b0405dd-af65-4ad8-b025-a62680aa6235/call-restore_pedigree_ids/uhs4_afr_relatedness.final.related_samples.remove . 
aws s3 cp s3://rti-cromwell-output/cromwell-execution/relatedness_wf/5b0405dd-af65-4ad8-b025-a62680aa6235/call-remove_fam_pedigree/uhs4_afr_relatedness.noped.idmap.fam .

    
wc -l *remove
#0 uhs4_eur_relatedness.final.related_samples.remove
#36 uhs4_afr_relatedness.final.related_samples.remove

awk '$9=="2nd_degree_relative"' uhs4_afr_relatedness.final.king.pruned.annotated.k0  | wc -l
# 24
awk '$9=="3+_degree_relative"' uhs4_afr_relatedness.final.king.pruned.annotated.k0  | wc -l
# 15
